In [3]:
import pandas as pd
import numpy as np
import os
from ete3 import NCBITaxa, PhyloTree
ncbi = NCBITaxa()
from collections import Counter
import gseapy as gp
from gseapy.plot import barplot, dotplot

In [4]:
table_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Tables\\'
fig_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Figures\\'
samples=pd.read_excel(table_path+'Table S1\\Table S1.TE_Evo_352_Species.xlsx')
samples.shape

(352, 7)

### Enrichment of TE-associated Genes

In [7]:
gene_sets={}
PlantTF_path='D:\\18.TE_Evolution\\03.Protein_Annotation\\PlantTF_Family\\'
for i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+"_PlantTF.csv" in os.listdir(PlantTF_path):
        df=pd.read_csv(PlantTF_path+specie_id+"_PlantTF.csv",sep='\t')
        for tf,tf_df in df.groupby("TF_Family"):
            genes=tf_df["Gene"].tolist()
            for gene in genes:
                if tf in gene_sets:
                    gene_sets[tf].append(gene)
                else:
                    gene_sets[tf]=[gene]

In [8]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part2.TE-associated_Genetic_Functions\\'
TEpep_genes_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
TE_dupgene_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
gene_list=[]
count=0
for  i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+".TEpep_to_Gene.txt" in os.listdir(TEpep_genes_path):
        gene_list+= [gene.strip() for gene in open(TEpep_genes_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]
    if specie_id+".TE_DupGenes.txt" in os.listdir(TE_dupgene_path):
        gene_list+=[gene.strip() for gene in open(TE_dupgene_path+specie_id+".TE_DupGenes.txt",'r').readlines()]
gene_list=list(set(gene_list))
count=len(gene_list)
print('TE-associated Genes',count)
if len(gene_list)>10:
    try:
        enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
            outdir=save_path,#输出目录
            top_term=20,
            cutoff=0.01#pvalue阈值
                                             )
        result=enr.results
        result=result.sort_values("Adjusted P-value")
        result['Title']=result['Term'].apply(lambda x:x.split(":")[1])
        result['Term']=result['Term'].apply(lambda x:x.split(":")[0])
        print(result.shape)
        result.to_excel(save_path+"TE-associated_Genes_PlantTF_Enrichment.xlsx",index=False)
    except:
        pass
else:
    print('Error: No TE-associated genes!')

TE-associated Genes 5871053


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


(2385, 9)


In [9]:
result[result["Adjusted P-value"]<0.01].shape

(88, 9)

### Enrichment of Transpoase Domesticated Genes

In [10]:
go_path='D:\\18.TE_Evolution\\03.Protein_Annotation\\GO\\'
TEpep_genes_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
gene_sets={}
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+".TEpep_to_Gene.txt"
    bg_genes={}
    if bg_file in os.listdir(TEpep_genes_path):
        for gene in [gene.strip() for gene in open(TEpep_genes_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]:
            bg_genes[gene]=1
    if specie_id+"_PlantTF.csv" in os.listdir(PlantTF_path):
        df=pd.read_csv(PlantTF_path+specie_id+"_PlantTF.csv",sep='\t')
        for tf,tf_df in df.groupby("TF_Family"):
            genes=tf_df["Gene"].tolist()
            for gene in genes:
                if gene in bg_genes:
                    if tf in gene_sets:
                        gene_sets[tf].append(gene)
                    else:
                        gene_sets[tf]=[gene]

0
100
200
300


In [11]:
gene_list_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Super_Family\\'
families=list(set([f.split(".TEpep")[0][:-5] for f in os.listdir(gene_list_path)]))
families

['Mutator',
 'Crypton',
 'Copia',
 'DDE_1',
 'CACTA',
 'LINE',
 'Gypsy',
 'piggybac',
 'P_element',
 'LTR_Roo',
 'hAT',
 'mariner_ant1',
 'Helitron',
 'Mariner',
 'ISb',
 'ISC1316']

In [12]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
for family in families:
    gene_list=[]
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        g_file=family+"_"+specie_id+'.TEpep_to_Gene.txt'#".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            gene_list+=genes
    print(family,len(gene_list))
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            result['Title']=result['Term'].apply(lambda x:x.split(":")[1])
            result['Term']=result['Term'].apply(lambda x:x.split(":")[0])
            print(family,result.shape)
            result.to_excel(save_path+family+"_PlantTF_TEpep_Dom_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)

Mutator 75681


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Mutator (372, 9)
Crypton 31
Crypton (10, 9)
Copia 354131


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Copia (552, 9)
DDE_1 1239
DDE_1 (87, 9)
CACTA 54024


...

LTR_Roo (182, 9)
hAT 33425


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


hAT (333, 9)
mariner_ant1 45
Helitron 16128


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Helitron (218, 9)
Mariner 744
Mariner (42, 9)
ISb 2
ISb
ISC1316 598
ISC1316 (91, 9)


### Enrichment of TE induced Dup Genes

In [13]:
bg_genes={}
gene_sets={}
background_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+'.TE_DupGenes.txt'#".TE_DupGenes.txt"
    #rm_file=specie_id+".TEpep_to_Gene.txt"
    if bg_file in os.listdir(background_path):
        genes=[gene.strip() for gene in open(background_path+bg_file,'r').readlines()]
        for gene in genes:
            bg_genes[gene]=1
        if specie_id+"_PlantTF.csv" in os.listdir(PlantTF_path):
            df=pd.read_csv(PlantTF_path+specie_id+"_PlantTF.csv",sep='\t')
            for tf,tf_df in df.groupby("TF_Family"):
                genes=tf_df["Gene"].tolist()
                for gene in genes:
                    if gene in bg_genes:
                        if tf in gene_sets:
                            gene_sets[tf].append(gene)
                        else:
                            gene_sets[tf]=[gene]

0
100
200
300


In [14]:
gene_list_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Super_Family\\'
families=list(set([f.split(".TE")[0][:-5] for f in os.listdir(gene_list_path)]))
families

['Mutator',
 'Copia',
 'CACTA',
 'Gypsy',
 'Tc1_Mariner',
 'hAT',
 'PIF-Harbinger',
 'Helitron']

In [ ]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
tepep_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
for family in families:
    gene_list=[]
    count=0
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        tepep_genes={}
        if specie_id+".TEpep_to_Gene.txt" in os.listdir(tepep_path):
            for gene in [gene.strip() for gene in open(tepep_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]:
                tepep_genes[gene]=1
        g_file=family+"_"+specie_id+".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            for gene in genes:
                if gene not in tepep_genes:
                    gene_list.append(gene)
                else:
                    count+=1
    print(family,len(gene_list),'TEpep Genes',count)
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            result['Title']=result['Term'].apply(lambda x:x.split(":")[1])
            result['Term']=result['Term'].apply(lambda x:x.split(":")[0])
            print(family,result.shape)
            result.to_excel(save_path+family+"_PlantTF_TE_DupGenes_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)

Mutator 2874488 TEpep Genes 194916


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Mutator (1853, 9)
Copia 1716190 TEpep Genes 271554
